In [1]:
from pyzotero import zotero
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.zotero import get_data_from_zotero
from code_utils.glutton import get_doi_glutton
from code_utils.utils import get_doi_cleaned,aplatir,wg_chap_to_dict,get_year_ipbes,check_doi_glutton
from code_utils.pickle import load_cache,write_cache
from code_utils.enriching_data_OpenAlex import get_countries_concepts_sdg,get_open_alex_data,get_open_alex_data_not_in_references

In [2]:
cached_openalex_data_ipbes = {}
cached_openalex_data_not_ipbes = {}

In [3]:
try:
    cached_openalex_data_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
    cached_openalex_data_not_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')
except:
    #write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
    #write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')
    print('oups')

5690 data in cached openalex data
58 data in cached openalex data


Read IPBES references in zotero files

In [ ]:
ZOTERO_KEY = os.getenv('ZOTERO_KEY')
IPBES_ZOTERO_ID = '2333077'
COLLECTION_IDS = ['8DQ8YFJI','JR5LKU4U','LBFNF62B','7IMYH9U3','DRZLUY9K','KTPTCAVF','UWNSUNXQ','BJGWRJAK','JT8FJXP6']
CHAPTER_IDS=['1','2.1','2.2','2.3','3','4','5','6','glossary']
ALL_COLLECTIONS={}

In [ ]:
ALL_COLLECTIONS=get_data_from_zotero(ZOTERO_KEY,IPBES_ZOTERO_ID,COLLECTION_IDS,CHAPTER_IDS,ALL_COLLECTIONS)

In [ ]:
df_ipbes_chap={}
for i in range(len(COLLECTION_IDS)):
    df_ipbes_chap[CHAPTER_IDS[i]]=pd.DataFrame(list(pd.DataFrame(ALL_COLLECTIONS[f'collection_chapter_{CHAPTER_IDS[i]}']).data))[['DOI','title','date','creators','tags']]
    df_ipbes_chap[CHAPTER_IDS[i]]['chap']=f'chap_{CHAPTER_IDS[i]}'
    print(f'{len(df_ipbes_chap[CHAPTER_IDS[i]][~pd.isna(df_ipbes_chap[CHAPTER_IDS[i]].DOI)])/len(df_ipbes_chap[CHAPTER_IDS[i]])*100} % doi available for chapter {CHAPTER_IDS[i]}')

Clean and enrich data 

In [ ]:
df_ipbes=pd.concat(list(df_ipbes_chap.values())).reset_index()
del df_ipbes['index']

In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

In [ ]:
df_ipbes.DOI.drop_duplicates()

In [ ]:
get_doi_glutton(df_ipbes.iloc[1,:])

In [ ]:
df_ipbes.loc[pd.isna(df_ipbes.DOI),'DOI']=df_ipbes.loc[pd.isna(df_ipbes.DOI),:].progress_apply(get_doi_glutton, axis=1)

In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

In [ ]:
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: get_doi_cleaned(x))
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_ipbes['freq']=1
df_ipbes=df_ipbes.rename(columns={'DOI':'doi'})

In [ ]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json', orient='records')

Enriching data through OpenAlex API integration

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json', orient='records')

In [ ]:
df_unique_doi=df_ipbes.groupby(by=['doi'], dropna=True).agg({'chap': lambda x: list(x), 'freq': 'sum'})
df_unique_doi=df_unique_doi.reset_index()

In [ ]:
#get data_OpenAlex from the API 
df_unique_doi.apply(lambda row: get_open_alex_data(cached_openalex_data_ipbes,row['doi']), axis= 1)

In [ ]:
len(cached_openalex_data_ipbes)

In [ ]:
write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json')

In [ ]:
df_unique_doi=df_ipbes['doi'].drop_duplicates().dropna()
df_unique_doi=pd.DataFrame({'doi':list(df_unique_doi)})

In [ ]:
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_names_list=[]
locations_ids_list=[]

for i,row in df_unique_doi.iterrows():
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_ids=get_countries_concepts_sdg(cached_openalex_data_ipbes,row,ipcc=True,i=0)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_names_list.append(locations_names)
    locations_ids_list.append(locations_ids)


df_unique_doi['countries'] = countries_list
df_unique_doi['concepts'] = concepts_list
df_unique_doi['sdg'] = sdg_list
df_unique_doi['topics'] = topics_list
df_unique_doi['is_OA_available'] = is_OA_available_list
df_unique_doi['year_OA'] = year_list
df_unique_doi['authors_name'] = names_list
df_unique_doi['rors'] = rors_list
df_unique_doi['institutions_names'] = institutions_names_list
df_unique_doi['locations_names'] = locations_names_list
df_unique_doi['locations_id'] = locations_ids_list

In [ ]:
df_unique_doi.is_OA_available.value_counts()

In [ ]:
df_unique_doi=df_unique_doi[df_unique_doi.is_OA_available]

In [ ]:
df_unique_doi=df_unique_doi.reset_index()
del df_unique_doi['index']

In [ ]:
df_unique_doi.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_OA.json', orient='records')

Ipbes references data

In [ ]:
df_unique_doi=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_OA.json', orient='records')

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json')

In [ ]:
df_ipbes=pd.merge(df_ipbes[['doi', 'title', 'date', 'creators', 'tags']], df_unique_doi, on='doi', how='inner')

In [ ]:
df_ipbes['year']=df_ipbes['date'].apply(lambda x: get_year_ipbes(x) if pd.isna(x)==False else None)

In [ ]:
df_ipbes['check']=df_ipbes.apply(lambda row: check_doi_glutton(row), axis=1)
df_ipbes.check.value_counts()

In [ ]:
df_ipbes[df_ipbes.check==False]

In [ ]:
df_ipbes=df_ipbes[(df_ipbes.check) & (pd.isna(df_ipbes['is_OA_available'])==False)]

In [ ]:
del df_ipbes['check']

In [ ]:
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)

In [ ]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_ipbes.jsonl', orient='records',lines=True)

Find the data for constructing the learning model ( IPBES related or not )

In [4]:
df_ipbes=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_ipbes.jsonl', lines= True).dropna(subset=['year'])

In [5]:
df_ipbes['year'] = pd.to_numeric(df_ipbes['year'], errors='coerce')
df_ipbes=df_ipbes.dropna(subset=['year','doi', 'title','topics','locations_names','locations_id']).drop_duplicates(subset=['doi'])
year=df_ipbes.drop_duplicates(subset='doi').dropna(subset=['doi'])['year'].dropna().sort_values().apply(lambda x: int(x))
year_counts = pd.Series(list(year)).value_counts().to_dict()

In [6]:
dois=list(df_ipbes.doi.dropna().drop_duplicates())

In [7]:
sum(list(year_counts.values()))

5531

In [10]:
year_counts_not_ipbes={}

In [ ]:
df_ipbes.topics.value_counts()[:50]

In [11]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipbes[year]=[]
    year_counts_not_ipbes[year]=0
    while year_counts_not_ipbes[year]<year_counts[year]:
        get_open_alex_data_not_in_references(dois,cached_openalex_data_not_ipbes,year_counts,year_counts_not_ipbes,year,ipcc="ipbes")
    cached_openalex_data_not_ipbes[year]=cached_openalex_data_not_ipbes[year][:year_counts[year]]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 283329
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 624 publications pour completer l'année 2016


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 288988
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 528 publications pour completer l'année 2016


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 292721
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


...

plus que 1 publications pour completer l'année 1976


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:1954&sample=200&per-page=200 HTTP/1.1" 200 146322
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 1 publications pour completer l'année 1954


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:1936&sample=200&per-page=200 HTTP/1.1" 200 129847


plus que 1 publications pour completer l'année 1936


In [12]:
sum(list(year_counts.values()))

5531

In [13]:
len(aplatir(list(cached_openalex_data_not_ipbes.values())))


5531

In [14]:
len(cached_openalex_data_not_ipbes)

58

In [15]:
write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')

In [16]:
df_not_ipbes=pd.DataFrame()
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_names_list=[]
locations_ids_list=[]
doi_list=[]

for k in range(len(aplatir(list(cached_openalex_data_not_ipbes.values())))):
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_ids=get_countries_concepts_sdg(cached_openalex_data=aplatir(list(cached_openalex_data_not_ipbes.values())),ipcc=False,i=k)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_names_list.append(locations_names)
    locations_ids_list.append(locations_ids)
    doi_list.append(doi)


df_not_ipbes['doi'] = doi_list
df_not_ipbes['title'] = title_list
df_not_ipbes['countries'] = countries_list
df_not_ipbes['concepts'] = concepts_list
df_not_ipbes['sdg'] = sdg_list
df_not_ipbes['topics'] = topics_list
df_not_ipbes['is_OA_available'] = is_OA_available_list
df_not_ipbes['year'] = year_list
df_not_ipbes['authors_name'] = names_list
df_not_ipbes['rors'] = rors_list
df_not_ipbes['institutions_names'] = institutions_names_list
df_not_ipbes['locations_names'] = locations_names_list
df_not_ipbes['locations_id'] = locations_ids_list

In [17]:
df_not_ipbes

,doi,title,countries,concepts,sdg,topics,is_OA_available,year,authors_name,rors,institutions_names,locations_names,locations_id
0,https://doi.org/10.1016/j.ijbiomac.2016.09.050,Potassium fulvate-modified graft copolymer of ...,[EG],"[Sorbent, Acrylic acid, Copolymer, Langmuir ad...","[{'id': '6', 'name': 'Clean water and sanitati...","[Advanced Cellulose Research Studies, Adsorpti...",True,2016,"[(Magdy F. Mohamed, [EG]), (Hisham Essawy, [EG...","[(https://ror.org/05fnp1145, EG), (https://ror...","[(Al-Azhar University, EG), (National Research...",International Journal of Biological Macromolec...,0141-8130
1,https://doi.org/10.21037/jtd.2016.s029,AB029. Clinical utility of a molecular diagnos...,[US],"[Medicine, Interstitial lung disease, Disease,...","[{'id': '3', 'name': 'Good health and well-bei...",[Interstitial Lung Diseases and Idiopathic Pul...,True,2016,"[(Xiaoping Wu, [US]), (Michael Rosenbluth, [US...","[(https://ror.org/02t11pf93, US), (https://ror...","[(Veracyte (United States), US), (Cornell Univ...",Journal of Thoracic Disease,2072-1439
2,https://doi.org/10.1515/ci-2016-0516,Guide (and Brief Guide) to Polymer Semiconductors,[None],"[Semiconductor, Polymer, Materials science, Po...",None,[Organic and Molecular Conductors Research],True,2016,[None],[None],[None],Chemistry International,0193-6484
3,https://doi.org/10.1093/pastj/gtw027,"Common Places and Private Spaces: Libraries, R...",[US],"[History, Genealogy]","[{'id': '11', 'name': 'Sustainable cities and ...",[History of Medicine Studies],True,2016,"[(Hannah Murphy, [US])]","[(https://ror.org/01m006041, US)]","[(Thiel College, US)]",Past & Present,0031-2746
4,https://doi.org/10.1093/gji/ggw067,Rock-magnetic and geochemical characteristics ...,[],"[Vertisol, Magnetic mineralogy, Geology, Maghe...","[{'id': '13', 'name': 'Climate action'}]","[Geomagnetism and Paleomagnetism Studies, Geol...",True,2016,"[(Neli Jordanova, []), (Diana Jordanova, [])]",[],[],Geophysical Journal International,0956-540X
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,https://doi.org/10.1021/ed051p623,A remote pipeting device,[],"[Citation, Altmetrics, Icon, Computer science,...",None,[Various Chemistry Research Topics],True,1974,"[(Dwight E. Talburt, []), (Dale A. Johnson, [])]",[],[],Journal of Chemical Education,0021-9584
5527,https://doi.org/10.7202/700546ar,Problématique de l’économie socialiste décentr...,"[CA, US]","[Gynecology, Political science, Medicine]","[{'id': '8', 'name': 'Decent work and economic...","[French Urban and Social Studies, Political an...",True,1975,"[(André Joyal, [CA, US])]","[(https://ror.org/02xrw9r68, CA), (https://ror...","[(Université du Québec à Trois-Rivières, CA), ...",Études internationales,0014-2123
5528,https://doi.org/10.1063/1.433097,Intramolecular 13C kinetic isotope effects in ...,[US],"[Intramolecular force, Chemistry, Kinetic isot...","[{'id': '6', 'name': 'Clean water and sanitati...","[Chemical Synthesis and Characterization, Zeol...",True,1976,"[(Richard W. Kidd, [US]), (Thomas Huang, [US])...","[(https://ror.org/047426m28, US), (https://ror...","[(University of Illinois Urbana-Champaign, US)...",The Journal of Chemical Physics,0021-9606
5529,https://doi.org/10.1042/bj0570297,The branched-chain fatty acids of butterfat. 4...,[NZ],"[Citation, Icon, Library science, Download, Bu...","[{'id': '9', 'name': 'Industry, innovation and...","[Peroxisome Proliferator-Activated Receptors, ...",True,1954,"[(R. P. Hansen, [NZ]), (F. B. Shorland, [NZ]),...",[],[],Biochemical Journal,0306-3283


In [18]:
df_not_ipbes.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_not_ipbes.jsonl', orient='records', lines= True)